In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
class MazeGame:
    def __init__(self):
        maze_layout = [[1, 1, 1, 1, 1, 1, 1],
               [1, 0, 0, 0, 0, 0, 1],
               [1, 0, 1, 1, 1, 0, 1],
               [1, 0, 1, 0, 0, 0, 1],
               [1, 0, 1, 1, 1, 1, 1],
               [1, 0, 0, 0, 0, 0, 0],
               [1, 1, 1, 1, 1, 1, 1]]
        """Initialize the maze game with a given layout."""
        self.layout = np.array(maze_layout)
        self.rgb_maze = self._to_rgb()
        self.start_pos = None
        self.exit_pos = None
        self.current_pos = None

    def _to_rgb(self):
        """Convert the maze layout to an RGB image."""
        rgb = np.zeros((*self.layout.shape, 3))  # Create a new RGB array
        for i in range(3):  # Copy the grayscale values into each RGB channel
            rgb[:, :, i] = 1 - self.layout
        return rgb

    def initialPosition(self, row, col):
        """Set the initial position in the maze."""
        if self.layout[row, col] == 0:
            self.start_pos = (row, col)
            self.current_pos = (row, col)
            self.set_point(row, col, [0, 0, 1])  # Mark initial position with blue
        else:
            print("Invalid initial position: It is on a wall.")

    def setExit(self, row, col):
        """Set the exit position in the maze."""
        if self.layout[row, col] == 0:
            self.exit_pos = (row, col)
            self.set_point(row, col, [0, 1, 0])  # Mark exit position with green
        else:
            print("Invalid exit position: It is on a wall.")

    def set_point(self, row, col, color):
        """Set a specific point in the maze to a given color."""
        self.rgb_maze[row, col] = color

    def makeMove(self, direction):
        """Attempt to move in the specified direction."""
        if self.current_pos is None:
            print("Initial position not set.")
            return 0

        row, col = self.current_pos
        if direction == "up":
            new_pos = (row - 1, col)
        elif direction == "down":
            new_pos = (row + 1, col)
        elif direction == "left":
            new_pos = (row, col - 1)
        elif direction == "right":
            new_pos = (row, col + 1)
        else:
            print("Invalid direction.")
            return 0

        if self.isValidMove(new_pos):
            self.current_pos = new_pos
            self.set_point(*self.start_pos, [1 - self.layout[self.start_pos]])  # Reset start position color
            self.start_pos = new_pos
            self.set_point(*new_pos, [0, 0, 1])  # Mark new position with blue
            if self.current_pos == self.exit_pos:
                # print("Success! You've found the way out.")
                return 1
        # else:
        #    print("Invalid move: Can't move through walls or out of bounds.")

    def isValidMove(self, pos):
        """Check if a move is valid (within bounds and not through walls)."""
        row, col = pos
        if 0 <= row < self.layout.shape[0] and 0 <= col < self.layout.shape[1]:
            return self.layout[row, col] == 0
        return False

    def plot(self):
        """Plot the maze."""
        plt.figure(figsize=(7, 7))
        plt.imshow(self.rgb_maze, interpolation='nearest')
        plt.xticks([]), plt.yticks([])  # Hide axes ticks
        plt.show()

In [ ]:
# Create an instance of MazeGame
maze_game = MazeGame()

# Set the initial and exit positions
maze_game.initialPosition(3, 3)
maze_game.setExit(5, 6)
maze_game.plot()

Model-based RL agent

In [ ]:
class ModelBasedAgent:
    def __init__(self, game, gamma=0.99):
        print('At initialization, the map is given as', game.layout.shape)
        self.game = game
        self.gamma = gamma  # Discount factor
        self.values = np.zeros(game.layout.shape)
        print('At initialization, the value matrix is given as,', self.values)
        self.policy = np.full(game.layout.shape, 'hold', dtype=object)
        print('At initialization, the policy matrix is given as,', self.policy)
        self.actions = ['up', 'down', 'left', 'right']

    # Bellman optimization to find the optimal action trajectory.
    def find_the_optimal_trajectory_simulation(self, iterations=100):
        for _ in range(iterations):
            new_values = np.copy(self.values)
            for row in range(self.game.layout.shape[0]):
                for col in range(self.game.layout.shape[1]):
                    if self.game.layout[row, col] == 0:  # Only update for navigable cells
                        state_value = []
                        for action in self.actions:
                            new_pos = self.predict_move((row, col), action)
                            reward = self.predict_reward((row, col), new_pos)
                            state_value.append(reward + self.gamma * self.values[new_pos])
                        best_action_value = max(state_value)
                        new_values[row, col] = best_action_value
                        self.policy[row, col] = self.actions[state_value.index(best_action_value)]
                        print('The action trajectory is updated to the following,')
            self.values = new_values

    def predict_move(self, pos, action):
        if action == "up":
            new_pos = (max(0, pos[0]-1), pos[1])
        elif action == "down":
            new_pos = (min(self.game.layout.shape[0]-1, pos[0]+1), pos[1])
        elif action == "left":
            new_pos = (pos[0], max(0, pos[1]-1))
        elif action == "right":
            new_pos = (pos[0], min(self.game.layout.shape[1]-1, pos[1]+1))
        return new_pos if self.game.layout[new_pos] == 0 else pos

    def predict_reward(self, pos, new_pos):
        if new_pos == self.game.exit_pos:
            return 100  # Large reward for reaching the exit
        elif new_pos == pos:
            return -5  # Penalty for hitting a wall
        else:
            return -1  # Small penalty for each move to encourage shortest path

    def run_policy(self):
        print("Running policy...")
        current_pos = self.game.current_pos
        steps = 0
        while current_pos != self.game.exit_pos:
            action = self.policy[current_pos]
            print(f"Step {steps}: at {current_pos} taking action {action}")
            self.game.makeMove(action)
            current_pos = self.game.current_pos
            steps += 1
            if steps > 50:  # Just in case something goes wrong
                print("Something went wrong. Stopping after 50 steps.")
                break

    def update_game(self, new_game):
        self.game = new_game

Simulation Run